<a href="https://colab.research.google.com/github/flee2-trinity/Deep-Learning-in-Sports-Result-Prediction-/blob/main/Rank2016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#add n days no win rate
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from google.colab import drive

drive.mount('/content/gdrive')
DATA_PATH = '/content/gdrive/My Drive/2021 Fall/Thesis I'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import datetime

def findFiles(path): 
  return glob.glob(path)

In [ ]:
import csv

filename = '/content/gdrive/My Drive/2021 Fall/Thesis I/NBA Data/nba 15-16/NatStat-NBA2016-Team_Statlines.csv'
with open(filename, newline='') as f:
      reader = csv.reader(f)
      """
      print(list(reader))
      ['GameDay', 'GameID', 'TeamID', 'Team', 'OpponentID', 'Opponent', 'Location',
       'Division', 'Conference', 'Playoffs', 'WinOrLoss', 'PlayerType', 'MIN', 'PTS', 
       'FGM', 'FGA', '3FM', '3FA', 'FTM', 'FTA', 'REB', 'AST', 'STL', 'BLK', 'OREB', 'TO', 'PF']
      """
      next(reader)
      rawDataL = list(reader)
print(rawDataL[0])

['2015-10-27', '31959', '6', 'Atlanta', '22', 'Detroit', 'H', '', 'Western', '', 'L', '', '234', '94', '37', '82', '8', '27', '12', '15', '40', '22', '9', '4', '7', '15', '25']


In [ ]:
filename18 = '/content/gdrive/My Drive/2021 Fall/Thesis I/NBA Data/nba 14-15/NatStat-NBA2015-Team_Statlines.csv'
with open(filename18, newline='') as f:
      reader = csv.reader(f)
      """
      print(list(reader))
      ['GameDay', 'GameID', 'TeamID', 'Team', 'OpponentID', 'Opponent', 'Location',
       'Division', 'Conference', 'Playoffs', 'WinOrLoss', 'PlayerType', 'MIN', 'PTS', 
       'FGM', 'FGA', '3FM', '3FA', 'FTM', 'FTA', 'REB', 'AST', 'STL', 'BLK', 'OREB', 'TO', 'PF']
      """
      next(reader)
      rawDataL18 = list(reader)
print(rawDataL18[0])

['2014-10-28', '30648', '1', 'Houston', '27', 'L.A. Lakers', 'V', '', 'Western', '', 'W', '', '234', '108', '31', '73', '12', '29', '34', '50', '47', '22', '7', '3', '14', '12', '30']


In [ ]:
import datetime

def readTime(ampm):
  t = 0
  tL = ampm.replace(' ',':').split(':')
  if tL[2] == "PM" and int(tL[0]) < 12:
    t += 12
  t += int(tL[0])
  t += int(tL[1])/60
  return t

seasonBegin = datetime.datetime(2015, 10, 26)
noonTime = readTime("12:00 PM")
#not using
class Stats:
  def __init__(self):
    """
    MIN, PTS,FGM, FGA, 3FM, 3FA, FTM, FTA, REB, AST, STL, BLK, OREB, TO, PF
    """
    self.MIN = 0
    self.PTS = 0
    self.FGM = 0
    self.FGA = 0
    self.FM3 = 0
    self.FA3 = 0
    self.FTM = 0
    self.FTA = 0
    self.REB = 0
    self.AST = 0
    self.STL = 0
    self.BLK = 0
    self.OREB = 0
    self.TO = 0
    self.PF = 0
    self.Win = 0
    self.Loss = 0
    self.WinRate = 0
  def update(self, MIN, PTS,FGM, FGA, FM3, FA3, FTM, FTA, REB, AST, STL, BLK, OREB, TO, PF):
    self.MIN = MIN
    self.PTS = PTS
    self.FGM = FGM
    self.FGA = FGA
    self.FM3 = FM3
    self.FA3 = FA3
    self.FTM = FTM
    self.FTA = FTA
    self.REB = REB
    self.AST = AST
    self.STL = STL
    self.BLK = BLK
    self.OREB = OREB
    self.TO = TO
    self.PF = PF
  def addWinRate(self, Win, Loss):
    self.Win = Win
    self.Loss = Loss
    self.WinRate = Win/(Win+Loss)
  def notEmpty(self):
    return self.MIN>0

class aGame:
  def __init__(self, GameDay, GameID, TeamID, Team, OpponentID, Opponent, Location,
       Division, Conference, Playoffs, WinOrLoss, PlayerType):
    self.GameID = int(GameID)
    self.TeamID = int(TeamID)
    gdL = GameDay.split("-")
    self.GameDay = datetime.datetime(int(gdL[0]), int(gdL[1]), int(gdL[2]))
    #self.GameTime = GameTime
    self.DayCount = (self.GameDay - seasonBegin).days
    self.Team = Team
    self.OpponentID = int(OpponentID)
    self.Opponent = Opponent
    self.Home = Location == 'H'
    self.Division = Division
    self.Conference = Conference
    self.Playoffs = Playoffs == 'Y'
    self.WinOrLoss = WinOrLoss == 'W'
    self.WinID = int(self.WinOrLoss)
    self.LossID = int(not self.WinOrLoss)
    self.PlayerType = PlayerType
    self.TStats = Stats()
    self.OppStats = Stats()
    self.TStatsP = Stats()
    self.OppStatsP = Stats()
    self.previous = False
  def printDay(self):
    print('Day', self.DayCount)
  def updateLastWRate(self, winRate):
    self.lastWinRate = winRate
def aLinetoGame(aline):
  newG = aGame(aline[0], aline[1], aline[2], aline[3], aline[4], aline[5], aline[6],
               aline[7], aline[8], aline[9], aline[10], aline[11])
  for n in range(12, 27):
    aline[n] = int(aline[n])

  newG.TStats.update(aline[12], aline[13], aline[14], aline[15], aline[16], aline[17], 
                     aline[18], aline[19], aline[20], aline[21], aline[22], aline[23], 
                     aline[24], aline[25], aline[26])
  return newG

def readLines(filename):
  with open(filename, newline='') as f:
      reader = csv.reader(f)
      next(reader)
      rawDataL = list(reader)
  #print(rawDataL)
  #'ID', 'GameDay', 'GameTime', 'Home', 'HomeID', 'Visitor', 'VisitorID', 'ScoreVis', 
  #'ScoreHome', 'GameStatus', 'Venue', 'CityState', 'Neutral', 'Division', 'Conference', 'Playoffs'
  #print("RawDataLength ",len(rawDataL))
  data = []
  print(rawDataL[0])
  for oneLine in rawDataL:
    g = aLinetoGame(oneLine)
    data.append(g)
  return data

In [ ]:
from copy import deepcopy
allData18 = readLines(filename18)
print(allData18[1])
print(len(allData18))
winRecord18 = []
lossRecord18 = []
for tg in allData18:
  tg.previous = True
  if tg.WinID and not tg.Playoffs:
    winRecord18.append(tg.TeamID)
  elif not tg.Playoffs:
    lossRecord18.append(tg.TeamID)
print(len(allData18))

['2014-10-28', '30648', '1', 'Houston', '27', 'L.A. Lakers', 'V', '', 'Western', '', 'W', '', '234', '108', '31', '73', '12', '29', '34', '50', '47', '22', '7', '3', '14', '12', '30']
2622
2622


In [ ]:
from collections import Counter
wr = Counter(winRecord18).most_common()
lr = Counter(lossRecord18).most_common()

In [ ]:
allData = readLines(filename)
print(allData[1])
print(len(allData))

['2015-10-27', '31959', '6', 'Atlanta', '22', 'Detroit', 'H', '', 'Western', '', 'L', '', '234', '94', '37', '82', '8', '27', '12', '15', '40', '22', '9', '4', '7', '15', '25']
2632


In [ ]:
tempD = allData[0:400]
tIDs = list(set(map(lambda x: x.TeamID, tempD)))
print(len(tIDs))

30


In [ ]:
from collections import defaultdict
dic = defaultdict(lambda: "Not Present")
for t in tIDs:
  w = list(filter(lambda x: x[0] == t, wr))[0][1]
  l = list(filter(lambda x: x[0] == t, lr))[0][1]
  dic[t] = w/(w+l)

In [ ]:
for tg in allData:
  previousGamesList = list(filter(lambda g: (g.DayCount < tg.DayCount) and (g.TeamID == tg.TeamID), allData))
  pMIN = 0
  pPTS = 0
  pFGM = 0
  pFGA = 0
  pFM3 = 0
  pFA3 = 0
  pFTM = 0
  pFTA = 0
  pREB = 0
  pAST = 0
  pSTL = 0
  pBLK = 0
  pOREB = 0
  pTO = 0
  pPF = 0
  pWin = 0
  pLoss = 0
  for p in previousGamesList:
    g = p.TStats
    pMIN += g.MIN
    pPTS += g.PTS
    pFGM += g.FGM
    pFGA += g.FGA
    pFM3 += g.FM3
    pFA3 += g.FA3
    pFTM += g.FTM
    pFTA += g.FTA
    pREB += g.REB
    pAST += g.AST
    pSTL += g.STL
    pBLK += g.BLK
    pOREB += g.OREB
    pTO += g.TO
    pPF += g.PF
    pWin += p.WinID
    pLoss += p.LossID
  gPlayed = len(previousGamesList)
  if gPlayed > 0:
    tg.TStatsP.update(pMIN/gPlayed, pPTS/gPlayed, pFGM/gPlayed, pFGA/gPlayed, pFM3/gPlayed,
                      pFA3/gPlayed, pFTM/gPlayed, pFTA/gPlayed, pREB/gPlayed, pAST/gPlayed,
                      pSTL/gPlayed, pBLK/gPlayed, pOREB/gPlayed, pTO/gPlayed, pPF/gPlayed)
    tg.TStatsP.addWinRate(pWin, pLoss)
  else:
    tg.TStatsP.update(234, 100, 44, 90, 10,
                      30, 20, 25, 45, 25,
                      7, 7, 11, 15, 22)
    tg.TStatsP.addWinRate(1, 1)
    #Random for the first game
allGames = []
for d in allData:
  d.updateLastWRate(dic[d.TeamID])
  sameGameList = list(filter(lambda g: g.GameID == d.GameID, allGames))
  if len(sameGameList) > 0:
    for g in sameGameList:
      if g.Home: 
        g.OppStats = deepcopy(d.TStats)
        g.OppStatsP = deepcopy(d.TStatsP)
      else:
        d.OppStats = deepcopy(g.TStats)
        d.OppStatsP = deepcopy(g.TStatsP)
        allGames.remove(g)
        allGames.append(d)
  else:
    allGames.append(d)
allData = allGames
print(len(allData))

1316


In [ ]:
print(len(allData18))
allGames18 = []
setGameID = set(map(lambda g: g.GameID, allData18))

for d in setGameID:
  #d.updateLastWRate(dic[d.TeamID])
  sameGame = list(filter(lambda g: g.GameID == d, allData18))[0]
  sameGame.previous = True
  allGames18.append(sameGame)
allData18 = list(filter(lambda g: not g.Playoffs, allGames18))
print(len(allData18))
allData18.sort(key = lambda g: g.GameDay)
print(len(allData18))

2622
1230
1230


In [ ]:
def sep2Groups(allD, afterN, oldData):#For Ranking
  allDataP = list(filter(lambda g:not g.Playoffs, allD))
  test_daysAfterN = list(filter(lambda g: g.DayCount >= afterN, allDataP))
  test_daysAfterN = list(map(lambda g: g.DayCount, test_daysAfterN))
  test_days = list(set(test_daysAfterN))
  test_days.sort()

  test_listGroups = []
  train_listGroups = []
  for daysC in test_days:
    tempL = list(filter(lambda g: g.DayCount < daysC, allDataP))
    train_listGroups.append(tempL + oldData)
    test_listGroups.append(list(filter(lambda g: g.DayCount >= daysC, allDataP)))
  return (train_listGroups, test_listGroups)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def gametoTensor(g : aGame):
  tensor = torch.zeros(1, 1, 82)
  
  tensor[0][0][g.TeamID] = 1
  tensor[0][0][g.OpponentID+37] = 1
  tensor[0][0][77] = g.DayCount/300
  tensor[0][0][78] = int(g.previous)
  tensor[0][0][79] = g.TStatsP.Win/72
  tensor[0][0][80] = g.TStatsP.Loss/72
  tensor[0][0][81] = g.TStatsP.WinRate
  #winloss
  #previous win loss
  
  return tensor

def gamestoTensorTrain(games, test = False):
  g_tensor_list = []
  currentWinRecord = []
  for g in games:
    g_tensor_list.append((gametoTensor(g), g.WinID))
    if g.WinID > 0 and not g.previous:
      currentWinRecord.append(g.TeamID)
    elif not g.previous:
      currentWinRecord.append(g.OpponentID)
  return g_tensor_list, currentWinRecord
def gamestoTensorTest(games, test = False):
  g_tensor_list = []
  for g in games:
    g_tensor_list.append((gametoTensor(g), g.WinID, g.TeamID, g.OpponentID))
  return g_tensor_list

In [ ]:
def allTeamsComb(day):
  rankTestList = []
  tempLine = ['2015-10-16', '985248', '5', 'Boston', '28', 'Philadelphia', 
                'H', '', '', '', 'W', '', '234', '103', '41', '96', '11', '37', 
                '10', '14', '54', '21', '7', '6', '12', '14', '20']
  for a in tIDs:
    for b in tIDs:
      if a != b:
        testG = aLinetoGame(tempLine)
        testG.TeamID = a
        testG.OpponentID = b
        testG.DayCount = day
        testG.updateLastWRate(dic[a])
        testG.TStatsP.addWinRate(1, 1)
        rankTestList.append((gametoTensor(testG), a, b))
  return rankTestList

In [ ]:

def getRankList(rankRecord):
  #print(len(rankRecord))
  rC = Counter(rankRecord).most_common()

  r = list(map(lambda x: x[0], rC))
  nRank = []
  for a in tIDs:
    if a not in r:
      nRank.append(30)
    else:
      nRank.append(r.index(a)+1)
  return nRank

In [ ]:
allRecord = []
nonPlayoffData = list(filter(lambda g:not g.Playoffs, allData))
print(len(allData))
for g in nonPlayoffData:
  if g.WinID> 0 :
    allRecord.append(g.TeamID)
  else:
    allRecord.append(g.OpponentID)
print(len(allRecord))
playOffRanking = getRankList(allRecord)
print(playOffRanking)

1316
1230
[18, 26, 10, 13, 7, 9, 23, 14, 3, 4, 6, 20, 27, 22, 8, 2, 11, 17, 24, 12, 15, 21, 1, 19, 29, 30, 16, 5, 28, 25]


In [ ]:
import time
import math
from statistics import mean
import scipy.stats as stats
from scipy.stats import kendalltau
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def runModel(train_listGroups, test_listGroups):
  start = time.time()
  train_tlistGroups = list(map(lambda group: gamestoTensorTrain(group), train_listGroups))
  test_tlistGroups = list(map(lambda group: gamestoTensorTest(group, test = True), test_listGroups))
  totalR_list = []
  print(len(train_tlistGroups), len(test_tlistGroups))
  for t in range(len(train_tlistGroups)):
    print('Model: ', t)
    model = nn.Sequential(nn.Linear(82, 64),
                          nn.Tanh(),
                          nn.Linear(64, 16),
                          nn.Tanh(),
                          nn.Linear(16, 2),
                          nn.LogSoftmax(dim=1))
    #model = model.to(device=device)
    learning_rate = 0.005
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    loss_fn = nn.NLLLoss()#meansquarerror
    n_epochs = 30
    currentRecord = train_tlistGroups[t][1]
    #print(len(currentRecord))
    for epoch in range(n_epochs):
      
      train_loader = torch.utils.data.DataLoader(train_tlistGroups[t][0], batch_size = 2, shuffle = True)
      for gs, labels in train_loader:
        #gs = gs.to(device=device)
        #labels = labels.to(device = device)
        batch_size = gs.shape[0]
        outputs = model(gs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      if (epoch+1)%10 == 0:
        print(timeSince(start))
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

    powerTestList = allTeamsComb(test_listGroups[t][0].DayCount)
    powerRankRecord = []
    for k, a, b in powerTestList:
      outputs = model(k.view(1, -1))
      #print(outputs)
      _, predicted = torch.max(outputs, dim=1)
      #print(predicted)
      if predicted == 1:
        powerRankRecord.append(a)
      else:
        powerRankRecord.append(b)
    powerRankList = getRankList(powerRankRecord)#Power Ranking
    #print(playOffRanking)
    powTau, _ = kendalltau(playOffRanking, powerRankList)
    print(powTau)
    scheduleRecord = []
    for gt, label, tID, oID in test_tlistGroups[t]:
      outputs = model(gt.view(1, -1))
      _, predicted = torch.max(outputs, dim=1)
      if predicted:
        scheduleRecord.append(tID)
      else:
        scheduleRecord.append(oID)
    #print(len(currentRecord+scheduleRecord))
    fakePlayOffRanking = getRankList(currentRecord+scheduleRecord)
    playTau, _ = kendalltau(playOffRanking, fakePlayOffRanking)
    print(playTau)
    totalR_list.append((t, powTau, playTau))
  print(timeSince(start))
  return totalR_list

In [ ]:
def writeInRank(newRecord):
  p1Name='/content/gdrive/My Drive/2021 Fall/Thesis I/Records/Rank2016.txt'
  p1file=open(p1Name)
  p1file_content = p1file.read()
  p1file=open(p1Name,"w")
  p1file.write(p1file_content)
  for d, pow, p in newRecord:
    p1file.write(str(d)+" "+ str(pow) + " " + str(p) + "\n")
  p1file.write("-\n")
  p1file=open(p1Name)
  p1file_content = p1file.read()
  #print(p1file_content)
  p1file.close()

In [ ]:
allData18.sort(key = lambda x: x.GameDay)
print(len(allData18))
trainGs, testGs = sep2Groups(allData, 5, allData18[-int(len(allData18)/4):])
print(len(trainGs), len(testGs))

1230
157 157


In [ ]:

for i in range(30):
  print('Run %d' %i)
  writeInRank(runModel(trainGs, testGs))

Streaming output truncated to the last 5000 lines.
21m 19s
Epoch: 19, Loss: 2.033677
21m 23s
Epoch: 29, Loss: 1.091258
0.8390804597701149
0.9586206896551724
21m 23s
Run 16
157 157
Model:  0
0m 14s
Epoch: 9, Loss: 0.683690
0m 15s
Epoch: 19, Loss: 0.738191
0m 16s
Epoch: 29, Loss: 0.546662
0.5172413793103449
0.5172413793103449
Model:  1
0m 17s
Epoch: 9, Loss: 0.779804
0m 18s
Epoch: 19, Loss: 0.732393
0m 19s
Epoch: 29, Loss: 0.269867
0.4850574712643679
0.5080459770114942
Model:  2
0m 20s
Epoch: 9, Loss: 0.687383
0m 21s
Epoch: 19, Loss: 0.559212
0m 22s
Epoch: 29, Loss: 0.689643
0.4482758620689655
0.5172413793103449
Model:  3
0m 24s
Epoch: 9, Loss: 0.672096
0m 25s
Epoch: 19, Loss: 0.749937
0m 26s
Epoch: 29, Loss: 0.385423
0.5816091954022989
0.6183908045977011
Model:  4
0m 27s
Epoch: 9, Loss: 0.721380
0m 28s
Epoch: 19, Loss: 0.714176
0m 29s
Epoch: 29, Loss: 0.655359
0.44367816091954027
0.4666666666666667
Model:  5
0m 30s
Epoch: 9, Loss: 0.671824
0m 31s
Epoch: 19, Loss: 0.635517
0m 32s
Epoch: 